## Install Dependencies

In [2]:
!pip install torch torchvision
!pip install opencv-python-headless opencv-python opencv-contrib-python
!pip install matplotlib
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


   ---------------------------------------- 0.0/39.4 MB ? eta -:--:--
   --------- ------------------------------ 9.4/39.4 MB 48.8 MB/s eta 0:00:01
   ---------------------- ----------------- 21.8/39.4 MB 52.8 MB/s eta 0:00:01
   ----------------------------------- ---- 34.9/39.4 MB 55.4 MB/s eta 0:00:01
   ---------------------------------------- 39.4/39.4 MB 52.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   ---------- ----------------------------- 12.6/46.2 MB 60.7 MB/s eta 0:00:01
   --------------------- ------------------ 24.9/46.2 MB 60.7 MB/s eta 0:00:01
   ------------------------------ --------- 35.4/46.2 MB 56.2 MB/s eta 0:00:01
   ---------------------------------------  46.1/46.2 MB 55.4 MB/s eta 0:00:01
   ---------------------------------------- 46.2/46.2 MB 51.5 MB/s eta 0:00:00
c:\Users\cucum\OneDrive\Documents\GitHub\image-classification\yolov5


Cloning into 'yolov5'...


## Load the YOLOv5 Model

In [ ]:
import torch  

model = torch.hub.load('.', 'yolov5s', source='local') 

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\cucum\\OneDrive\\Documents\\GitHub\\image-classification\\yolov5\\ultralytics/yolov5\\hubconf.py'